In [ ]:
!pip install datasets transformers accelerate evaluate
#this line of code from the fine-tuning model screws up without upgrading sympy:
# model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
!pip install --upgrade sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
import random

# Load the dataset
all_data = load_dataset("CreativeLang/ColBERT_Humor_Detection").filter(lambda
example: len(example["text"])<1024)
random.seed(1)

# Receiving an error that the sample size is larger than the data. These print statements shows the data's format
# print(all_data)
# print(all_data["train"])
# print(all_data["train"]["humor"])
# print(all_data["train"]["text"])

# Load our small sample of data for training and analysis
# After looking at the data format, all_data["train"]["text"] seems more promising than all_data
random_train_indices = random.sample(range(len(all_data["train"]["text"])), 18000)
train_data = all_data["train"].select(random_train_indices)
random_eval_indices = random.sample(range(len(all_data["train"]["humor"])), 2000)
eval_data = all_data["train"].select(random_eval_indices)

# I commented out the following two lines of code because there are only two data splits, and they both exist inside of the sole train dataset
# random_test_indices = random.sample(range(len(all_data)), 2000)
# test_data = all_data.select(random_test_indices)

# Print the number of rows in each subset to verify
print(f"Train data: {train_data}")
print(f"Eval data: {eval_data}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.70k [00:00<?, ?B/s]

dataset.csv:   0%|          | 0.00/14.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/200000 [00:00<?, ? examples/s]

Train data: Dataset({
    features: ['text', 'humor'],
    num_rows: 18000
})
Eval data: Dataset({
    features: ['text', 'humor'],
    num_rows: 2000
})


In [ ]:
train_data[16]["text"]

"Sleepy meerkat wants to stay awake, can't (video)"

In [ ]:
# Regex classifier
import re

def regex_classifier(text):
  return bool(re.match(r"(?i)^.*(\?\s+.*|why|what|how|who|did|does|is|are|can|could|would|should).*[.!]$",text))

# Find jokes to make sure the classifier works relatively well
# found = 0
# i = 0
# while found < 5 and i <= len(train_data)-1:
#   message = (regex_classifier(train_data[i]["text"]))
#   message_txt = train_data[i]["text"]
#   if message != None:
#     found += 1
#     print(message)
#     print(message_txt)
#     print()
#   i+=1

In [ ]:
# We test our classifier with this function

from sklearn.metrics import classification_report
# y_true & y_pred are arrays of labels and predictions respectively
def evaluate_performance(y_true, y_pred):
    """Create and print the classification report"""
    performance = classification_report(
        y_true, y_pred
    )
    print(performance)

In [ ]:
# Run the regex classifier to get an array of predictions
y_pred = []
for i in range(len(eval_data["text"])):
  y_pred.append(regex_classifier(eval_data[i]["text"]))
#format the y_true values by switching true/false to joke/text
# y_true = ["joke" if label else "text" for label in eval_data["humor"]]
# Change 'text' and 'joke' to false and true, respectively
y_true = [True if label else False for label in eval_data["humor"]]

In [ ]:
# confirming the shape of my data
#print(eval_data)
#eval_data["text"]
#print(y_pred)
#print(eval_data[0]["text"])
#print(y_pred[0])
#print(eval_data["humor"])

# print(y_pred)
# print(y_true)

for i in range(len(y_true)):
  if y_true[i] == True:
    print(eval_data[i]["text"])

Went to a zoo that only had one dog it was a shitzoo
You are what you eat.. funny, i dont rember eating a sexy beast this morning.
Why does a pencil shave? to look sharp
So bored i just logged into my linkedin account.
Hey daddy!!! watch what i can do!! - aaaaaaaaand hospital.
Who do you pat with encouragement when they fail to work? a remote control
Unfortunately..... nobody wants to have sex with your inner beauty.
Three beans travelled around australia together, where was their final destination? in cairns.
Jokes what do you call a mexican in a car
I only smoke for the health tips from complete motherfucking strangers
I guess you could say caitlyn jenner is... transjennered.
Why did hitler make such a bad weatherman? even when it was rain, he kept yelling heil!
Want to hear a joke about nitric oxide? sodium hypobromite
I have never turned down an offer to smell something
Whom hath released the hounds? whom? whom? whom? whom?
Oh great. i forgot to pack an apple in my lunch and now th

In [ ]:
# Results of regex classifier

evaluate_performance(y_true, y_pred)

              precision    recall  f1-score   support

       False       0.70      0.98      0.82      1022
        True       0.97      0.55      0.70       978

    accuracy                           0.77      2000
   macro avg       0.83      0.77      0.76      2000
weighted avg       0.83      0.77      0.76      2000



In [ ]:
# Logistic Regression model

# acquire the necessary tools
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# just so I can see the data
#train_data[10]

In [ ]:
# create features for unigrams and bigrams
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_train = vectorizer.fit_transform(train_data["text"])
X_test = vectorizer.transform(eval_data["text"])

In [ ]:
# create the logistic regression and apply it with the data's actual labels
model = LogisticRegression(verbose=1)
model.fit(X_train, train_data["humor"])

LogisticRegression(verbose=1)

In [ ]:
# this is where we see how the model did
predicted = model.predict(X_test)
print(classification_report(eval_data["humor"], predicted))
# false indicates the lack of humor while true indicates its presence

              precision    recall  f1-score   support

       False       0.93      0.90      0.91      1022
        True       0.90      0.93      0.91       978

    accuracy                           0.91      2000
   macro avg       0.91      0.91      0.91      2000
weighted avg       0.92      0.91      0.91      2000



In [ ]:
# This snippet will find examples that the logistic regression got right but the regex got wrong
counter = 0

for i, output in enumerate(predicted):
  if output != y_pred[i]:
    if eval_data[i]["humor"] == output:
      counter += 1
      print(f"Review Text: {eval_data[i]['text']}")
      print(f"True Label & Logistic Regression Label: {eval_data[i]['humor']}")
      print(f"Regex Label: {y_pred[i]}")
      print("-" * 20)

      if counter >= 3:
        break

Review Text: Went to a zoo that only had one dog it was a shitzoo
True Label & Logistic Regression Label: True
Regex Label: False
--------------------
Review Text: Why does a pencil shave? to look sharp
True Label & Logistic Regression Label: True
Regex Label: False
--------------------
Review Text: So bored i just logged into my linkedin account.
True Label & Logistic Regression Label: True
Regex Label: False
--------------------


The first example doesn't follow a traditional joke format. The only indication of the joke is the fact that a Shih Tzu is a type of dog, but it's not even spelled correctly. The second example lacks proper punctuation and the third is hardly a joke.

**Classifier 1 - Regex**

This classifier is as simple as it gets. I have one regex condition - The text must be a question followed by a sentence that ends with a period or exclamation mark. The question sentence does not have to be there if there is a question word. This mimics the "setup, punchline" style of joke that is found often in the data.

**Classifier 2 - Logistic Regression & IDF**

The only interesting setting that I played around with on the logistic regression model was the vectorizer's n-gram range. TF-IDF tends to be very sparse data, so it made sense to give the model more features to interpret. The two different n-gram sizes contain more data than the unigrams or bigrams could carry alone.

**Choice of metric**

I chose to go with f1-score for my metric of evaluation. Accuracy is fun and all, but this dataset seemed to have a lot more normal text than humor at first glance, and accuracy doesn't work very well on data that is so unbalanced since my classifier could just predict non-humor all the time and be mostly accurate. Neither preicision nor recall are more important than one another for this classifier, so I decided f1-score was a nice balance.

ZERO SHOT

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load an instruction-tuned decoder model
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def classify_text(text):
    prompt = f"""Classify the following sentence as either "Humorous" or "Not Humorous".

Sentence: "{text}"
Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=10)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Normalize prediction to 0/1
    if "humorous" in prediction.lower():
        return 1
    else:
        return 0


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
y_pred = [classify_text(text) for text in eval_data["text"]]
evaluate_performance(eval_data["humor"], y_pred)


              precision    recall  f1-score   support

       False       0.00      0.00      0.00      1022
        True       0.49      1.00      0.66       978

    accuracy                           0.49      2000
   macro avg       0.24      0.50      0.33      2000
weighted avg       0.24      0.49      0.32      2000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


FEW SHOT

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Few-shot examples
few_shot_prompt = """
Classify whether the following sentences are "Humorous" or "Not Humorous".

Example 1:
Sentence: "Why don't skeletons fight each other? They don't have the guts."
Answer: Humorous

Example 2:
Sentence: "I had cereal for breakfast."
Answer: Not Humorous

Example 3:
Sentence: "Parallel lines have so much in common. It’s a shame they’ll never meet."
Answer: Humorous

Now classify this sentence:
Sentence: "{sentence}"
Answer:
"""

def classify_text(text):
    prompt = few_shot_prompt.format(sentence=text)
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=10)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()
    return 1 if "humorous" in prediction else 0


In [ ]:
y_pred = [classify_text(text) for text in eval_data["text"]]
evaluate_performance(eval_data["humor"], y_pred)


              precision    recall  f1-score   support

       False       0.00      0.00      0.00      1022
        True       0.49      1.00      0.66       978

    accuracy                           0.49      2000
   macro avg       0.24      0.50      0.33      2000
weighted avg       0.24      0.49      0.32      2000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


98% accuracy model

In [ ]:
# retrieve an encoder so we can fine-tune it

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load Model and Tokenizer
model_id = "prajjwal1/bert-tiny"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
# tokenize input so the model can work with it

from transformers import DataCollatorWithPadding

# Pad to the longest sequence in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
   """Tokenize input data"""
   return tokenizer(examples["text"], truncation=True)

# Tokenize train/test data
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = eval_data.map(preprocess_function, batched=True)

# The trainer needs the humor to be called "labels"
def add_labels(example):
    example["labels"] = int(example["humor"])
    return example

tokenized_train = tokenized_train.map(add_labels)
tokenized_test = tokenized_test.map(add_labels)

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# I want to view the inputs
print(tokenized_train)
#print(tokenized_test)

Dataset({
    features: ['text', 'humor', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 18000
})


In [ ]:
# we give the model the metric on which to operate

import numpy as np
import evaluate

def compute_metrics(eval_pred):
    """Calculate F1 score"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    load_f1 = evaluate.load("f1")
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"f1": f1}

In [ ]:
# we finally get to train our model

from transformers import TrainingArguments, Trainer

# Training arguments for parameter tuning
training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=1,
   weight_decay=0.01,
   save_strategy="epoch",
   report_to="none"
)

# Trainer which executes the training process
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

<ipython-input-35-fb29aa648f15>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


This was a tiny version of bert that I found. I got training time to go down to 3 minutes from 4 hours

In [ ]:
trainer.train()

Step,Training Loss
500,0.437600
1000,0.240900


TrainOutput(global_step=1125, training_loss=0.32604754299587674, metrics={'train_runtime': 153.5404, 'train_samples_per_second': 117.233, 'train_steps_per_second': 7.327, 'total_flos': 1172302213440.0, 'train_loss': 0.32604754299587674, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.19524560868740082,
 'eval_f1': 0.943943943943944,
 'eval_runtime': 4.2187,
 'eval_samples_per_second': 474.08,
 'eval_steps_per_second': 29.63,
 'epoch': 1.0}

Those are the results of the fine-tuning on our tiny BERT. Now we want to try freeze-training the model

In [ ]:
# Load Model and Tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Print layer names
for name, param in model.named_parameters():
    print(name)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

In [ ]:
# Now we freeze every layer besides the classifier head itself

for name, param in model.named_parameters():

     # Trainable classification head
     if name.startswith("classifier"):
        param.requires_grad = True

      # Freeze everything else
     else:
        param.requires_grad = False

In [ ]:
# We can check whether the model was correctly updated
for name, param in model.named_parameters():
     print(f"Parameter: {name} ----- {param.requires_grad}")

Parameter: bert.embeddings.word_embeddings.weight ----- False
Parameter: bert.embeddings.position_embeddings.weight ----- False
Parameter: bert.embeddings.token_type_embeddings.weight ----- False
Parameter: bert.embeddings.LayerNorm.weight ----- False
Parameter: bert.embeddings.LayerNorm.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.query.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.query.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.key.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.key.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.value.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.value.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.weight ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.LayerNorm.weight ----- False
Parameter: bert.encoder.layer.0.attention.output

We see that only the classifier weights and bias are True (meaning they can be trained)

In [ ]:
from transformers import TrainingArguments, Trainer

# Trainer which executes the training process
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)
trainer.train()

<ipython-input-42-1da7f8eeebcf>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.676300
1000,0.650200


TrainOutput(global_step=1125, training_loss=0.6613281724717882, metrics={'train_runtime': 46.3779, 'train_samples_per_second': 388.116, 'train_steps_per_second': 24.257, 'total_flos': 1172302213440.0, 'train_loss': 0.6613281724717882, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6272009015083313,
 'eval_f1': 0.8398645379777455,
 'eval_runtime': 3.6681,
 'eval_samples_per_second': 545.238,
 'eval_steps_per_second': 34.077,
 'epoch': 1.0}

These are the results for the frozen classification model (the only thing that was trained was the classification head).